In [1]:
%reload_ext autoreload
%autoreload 2

In [1]:
from depsurf import DepKind, VersionGroup, gen_report
from pathlib import Path

deps = sorted(set(DepKind.FUNC(f.name) for v in VersionGroup.REGULAR for f in v.img.func_groups.iter_funcs()))

data = gen_report(deps, VersionGroup.REGULAR, Path("/dev/null"))

[       groups.py:68 ] INFO: Loading funcs from /Users/szhong/Code/DepSurf/data/funcs/4.4.0-21-generic-amd64.jsonl
[       groups.py:68 ] INFO: Loading funcs from /Users/szhong/Code/DepSurf/data/funcs/4.8.0-22-generic-amd64.jsonl
[       groups.py:68 ] INFO: Loading funcs from /Users/szhong/Code/DepSurf/data/funcs/4.10.0-19-generic-amd64.jsonl
[       groups.py:68 ] INFO: Loading funcs from /Users/szhong/Code/DepSurf/data/funcs/4.13.0-16-generic-amd64.jsonl
[       groups.py:68 ] INFO: Loading funcs from /Users/szhong/Code/DepSurf/data/funcs/4.15.0-20-generic-amd64.jsonl
[       groups.py:68 ] INFO: Loading funcs from /Users/szhong/Code/DepSurf/data/funcs/4.18.0-10-generic-amd64.jsonl
[       groups.py:68 ] INFO: Loading funcs from /Users/szhong/Code/DepSurf/data/funcs/5.0.0-13-generic-amd64.jsonl
[       groups.py:68 ] INFO: Loading funcs from /Users/szhong/Code/DepSurf/data/funcs/5.3.0-18-generic-amd64.jsonl
[       groups.py:68 ] INFO: Loading funcs from /Users/szhong/Code/DepSurf/d

In [2]:
# load data
import pickle
with open("html/data.pkl", "rb") as f:
    data = pickle.load(f)

In [25]:
from utils import *
from depsurf import IssueEnum, VersionGroup

from collections import defaultdict


path = Path("html")
path.mkdir(exist_ok=True)

partitioned_results = defaultdict(dict)

for dep, report in data.items():
    result = {}
    for (group, version), issues in report.items():
        v = version.short_version
        if IssueEnum.FULL_INLINE in issues:
            result[v] = "F"
        elif IssueEnum.PARTIAL_INLINE in issues:
            result[v] = "P"
        elif IssueEnum.ABSENT in issues:
            result[v] = "-"
        else:
            result[v] = ""
    
    key = dep.name.replace("_", "").lower()
    partitioned_results[key[:1]][(key, dep.name)] = result


partitioned_results = {k: sorted(v.items()) for k, v in partitioned_results.items()}
                       


In [35]:

for char, results in partitioned_results.items():
    with open(path / f"func-{char}.html", "w") as f:
        f.write("<html>\n")
        f.write("<head>\n")
        f.write(f"<title>Kernel Function Inline Status - {char}</title>\n")
        f.write('<link rel="stylesheet" href="style.css">\n')
        f.write("</head>\n")

        f.write("<body>\n")
        f.write("<table>\n")
        f.write("<thead>\n")
        
        f.write("<tr>\n")  
        f.write(f'<td><span class="f">&nbsp;F&nbsp;</span>: Fully Inlined. <span class="p">&nbsp;P&nbsp;</span>: Partially Inlined. <span class="m">&nbsp;-&nbsp;</span>: Absent</td>\n')          
        for version in VersionGroup.REGULAR:
            f.write(f"<th>{version.short_version}</th>")
        f.write("</tr>\n")
        f.write("</thead>\n")

        f.write("<tbody>\n")
        for (_, dep), result in results:
            f.write(f'<tr><th id="{dep}"><pre>{dep}</pre></th>')
            for v in result.values():
                class_name = {
                    "F": "f",
                    "P": "p",
                    "-": "m",
                }.get(v)
                if class_name is not None:
                    f.write(f'<td class="{class_name}">{v}</td>')
                else:
                    f.write(f'<td>{v}</td>')
            f.write("</tr>\n")
        f.write("</tbody>\n")

        f.write("</table>\n")
        f.write("</body>\n")
        f.write("</html>\n")